# CIFAR10を用いた物体認識（ResNet）


---
## 目的

畳み込みニューラルネットワークのモデルとして，ResNetを用いて実験を行い，その構造を理解する．

## 準備

### Google Colaboratoryの設定確認・変更
本チュートリアルではPyTorchを利用してニューラルネットワークの実装を確認，学習および評価を行います．
**GPUを用いて処理を行うために，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．**


## モジュールのインポート
はじめに必要なモジュールをインポートする．

### GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`GPU availability: True`と表示されれば，GPUを使用した計算をChainerで行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．


In [ ]:
# モジュールのインポート
from time import time
import numpy as np
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

import torchsummary

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## データセットの読み込みと確認

学習データ（CIFAR10データセット）を読み込みます．

In [ ]:
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=1),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor()])
transform_test = transforms.Compose([transforms.ToTensor()])

train_data = torchvision.datasets.CIFAR10(root="./", train=True, transform=transform_train, download=True)
test_data = torchvision.datasets.CIFAR10(root="./", train=False, transform=transform_test, download=True)

## ネットワークモデルの定義
Residual Network (ResNet) を定義します．

ResNetはBottleneckと呼ばれる構造から構成されています．
まず，`BottleNeck(nn.Module)`で，任意の形のBottleNeckを定義できるクラスを作成します．
`__init__`関数の引数である，`in_planes`は入力される特徴マップのチャンネル数，`planes`はBottleNeck内の特徴マップのチャンネル数を指定します．

また，層を定義する`__init__`内では，`nn.Sequential()`という関数が用いられています．
これは，複数の層が格納されたリストを引数として受け取り，これらの層をひとまとめにしたオブジェクト（層）を定義する関数です・
下の関数では，畳み込みやBatchNormalizationがリスト内にされています．
`nn.Sequential`で定義した層`self.convs`では，実際に演算する際，すなわち`formward()`関数内では，`self.convs(x)`とすることで，リストに格納した演算をその順番通りに処理して返すことができます．

上で定義したBottleNeck構造を活用して，ResNet（ここではResNet50）を定義します．
`ResNet`クラス内で定義されている`self._make_layer()`は，任意の形（総数）のResidual Block (複数のBottleNeck構造からなる層)を定義します．
Residual Blockに入力されるチャンネル数`planes`，BottleNeckの数`num_blocks`，畳み込みのストライド`stride`を指定します．
その後，それらの引数に従い，指定した数・パラメータのBottleNeckをリストないに格納します．
最後に，上で説明した`nn.Sequential`を用いて一塊の層として定義し，返すことで，任意の数の層を持つresidual blockを定義します．

この`_make_layer()`を用いて，`__init__`でResNet全体を定義します．
`AdaptiveAvgPooling()`は任意のサイズの特徴マップに対して平均プーリングを適用する層です．
引数に`(1, 1)`を指定することで，どのようなサイズの特徴マップが入力された場合でも，1×1の特徴マップになるように平均プーリングを行います．




In [ ]:
class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.convs = nn.Sequential(*[nn.Conv2d(in_planes, planes, kernel_size=1, bias=False),
                                     nn.BatchNorm2d(planes),
                                     nn.ReLU(inplace=True),
                                     nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False),
                                     nn.BatchNorm2d(planes),
                                     nn.ReLU(inplace=True),
                                     nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False),
                                     nn.BatchNorm2d(self.expansion * planes)])

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.convs(x)
        out += self.shortcut(x)
        out = self.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, n_class=10, n_blocks=[3, 4, 6, 3]):
        super().__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        
        self.res1 = self._make_layer(64, n_blocks[0], stride=1)
        self.res2 = self._make_layer(128, n_blocks[1], stride=2)
        self.res3 = self._make_layer(256, n_blocks[2], stride=2)
        self.res4 = self._make_layer(512, n_blocks[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(2048, n_class)

    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(BottleNeck(self.in_planes, planes, stride))
            self.in_planes = planes * BottleNeck.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        h = self.relu(self.bn1(self.conv1(x)))
        h = self.res1(h)
        h = self.res2(h)
        h = self.res3(h)
        h = self.res4(h)
        h = self.avgpool(h)
        h = torch.flatten(h, 1)
        h = self.fc(h)
        return h
        
        
class ResNet50(ResNet):
    def __init__(self, n_class=10):
        super(ResNet50, self).__init__(n_class, n_blocks=[3, 4, 6, 3])

## ネットワークの作成
上のプログラムで定義したネットワークを作成します．

CNNクラスを呼び出して，ネットワークモデルを定義します． また，GPUを使う場合（use_cuda == True）には，ネットワークモデルをGPUメモリ上に配置します． これにより，GPUを用いた演算が可能となります．

学習を行う際の最適化方法としてモーメンタムSGD(モーメンタム付き確率的勾配降下法）を利用します． また，学習率を0.01，モーメンタムを0.9として引数に与えます．

最後に，定義したネットワークの詳細情報を`torchsummary.summary()`関数を用いて表示します．

In [ ]:
model = ResNet50(n_class=10)
if use_cuda:
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# モデルの情報を表示
torchsummary.summary(model, (3, 32, 32))

## 学習
１回の誤差を算出するデータ数（ミニバッチサイズ）を128，学習エポック数を100とします．
CIFAR10の学習データサイズを取得し，１エポック内における更新回数を求めます．
学習モデルに`image`を与えて各クラスの確率yを取得します．各クラスの確率yと教師ラベル`label`との誤差をsoftmax coross entropy誤差関数で算出します．
また，認識精度も算出します．そして，誤差をbackward関数で逆伝播し，ネットワークの更新を行います．

In [ ]:
# ミニバッチサイズ・エポック数の設定
batch_size = 128
epoch_num = 10
n_iter = len(train_data) / batch_size

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)

# 誤差関数の設定
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model.train()

start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    
    for image, label in train_loader:
        if use_cuda:
            image = image.cuda()
            label = label.cuda()

        y = model(image)
        loss = criterion(y, label)
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()
        
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed_time :{}".format(epoch,
                                                                                 sum_loss / n_iter,
                                                                                 count.item() / len(train_loader),
                                                                                 time() - start))

## テスト
学習したネットワークモデルを用いて評価を行います．

In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model.eval()

# 評価の実行
count = 0
with torch.no_grad():
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / 10000.))

## 課題


### 1. 学習の設定を変更し，認識精度の変化を確認しましょう．

**ヒント：プログラムの中で変更で切る設定は次のようなものが存在します．**
* ミニバッチサイズ
* 学習回数（Epoch数）
* 学習率
* 最適化手法
  * `torch.optim.Adagrad()`や`torch.optim.Adam()`などが考えられます．
  * PyTorchで使用できる最適化手法は[こちらのページ](https://pytorch.org/docs/stable/optim.html#algorithms)にまとめられています．


### 2. Data Augmentationの種類を追加して学習を行いましょう．

**ヒント**
：学習時に使用するData Augmentationは`transform_train`の部分で変更できます．

```python
transform_train = transforms.Compose([(この部分に使用するAugmentationの処理を追加) ,
                                      transforms.ToTensor(),
                                      transforms.LinearTransformation(Z, mean)])
```

PyTorch（torchvision）で使用可能な変換は[こちらのページ](https://pytorch.org/docs/stable/torchvision/transforms.html)にまとめられています．
